In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup # for scraping 2016 results
import urllib
from collections import Counter
import vincent # for visualizations
import json
vincent.core.initialize_notebook()

In [2]:
# using Python 3.5 for this project
import sys
print(sys.version)

3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


# Flow

- [Download and preprocess county-level results for 2012](#Guardian-data)
- [Download and preprocess county-level results for 2016](#Townhall-data)
- [Downlaod and preprocess county-level metadata](#Census-data)
- [Combine datasets](#Combine-data)
- [Export county-level results](#Export-data)
- [Visualize](#Visualize-data)

## Guardian data

In [3]:
# import Guardian data
# https://www.theguardian.com/news/datablog/2012/nov/07/us-2012-election-county-results-download#data
# NOTE: xlrd package needed (not installed by deafault) 
# to read data and format information from older Excel files (ie: .xls)
# get all data within the 'FULL DATA' worksheet and pass it into a DataFrame
guardian_full = pd.read_excel('http://image.guardian.co.uk/sys-files/Guardian/documents/2012/11/14/US_elect_county.xls',
                     sheetname = 'FULL DATA',
                     header = 0,
                     index_col=None,
                     convert_float = True,
                     converters={'FIPS Code':str},
                     parse_cols = "A, D, E, K, M, T, Y, AF, AK, AR, AW, BD, BI, BP")
                     
guardian_full.head()

WARNING *** file size (3825153) not 512 + multiple of sector size (512)


,State Postal,FIPS Code,County Name,TOTAL VOTES CAST,Party,Votes,Party.1,Votes.1,Party.2,Votes.2,Party.3,Votes.3,Party.4,Votes.4
0,AK,0,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN
1,AK,2000,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN
2,AL,0,Alabama,2064699,Dem,793620,GOP,1252453,Ind,2961.0,Ind,12283.0,Ind,3382.0
3,AL,1001,Autauga,23909,Dem,6354,GOP,17366,Ind,31.0,Ind,136.0,Ind,22.0
4,AL,1003,Baldwin,84988,Dem,18329,GOP,65772,Ind,122.0,Ind,597.0,Ind,168.0


In [4]:
# view by state
guardian_full[(guardian_full['State Postal'] == 'DC')]

,State Postal,FIPS Code,County Name,TOTAL VOTES CAST,Party,Votes,Party.1,Votes.1,Party.2,Votes.2,Party.3,Votes.3,Party.4,Votes.4
456,DC,0,District of Columbia,243348,Dem,222332,GOP,17337,Lib,1634.0,DCG,2045.0,NaN,NaN
457,DC,11001,District of Columbia,243348,Dem,222332,GOP,17337,Lib,1634.0,DCG,2045.0,NaN,NaN


In [5]:
# return results for only D & R
def dem_func(x):
    if x['Party'] == 'Dem':
        return x['Votes']
    elif x['Party.1'] == 'Dem':
        return x['Votes.1']
    elif x['Party.2'] == 'Dem':
        return x['Votes.2']
    elif x['Party.3'] == 'Dem':
        return x['Votes.3']
    elif x['Party.4'] == 'Dem':
        return x['Votes.4']

def gop_func(x):
    if x['Party'] == 'GOP':
        return x['Votes']
    elif x['Party.1'] == 'GOP':
        return x['Votes.1']
    elif x['Party.2'] == 'GOP':
        return x['Votes.2']
    elif x['Party.3'] == 'GOP':
        return x['Votes.3']
    elif x['Party.4'] == 'GOP':
        return x['Votes.4']

In [6]:
# get results for D & R only
guardian = guardian_full
guardian['votes_dem_2012'] = guardian.apply(lambda x: dem_func(x), axis=1)
guardian['votes_gop_2012'] = guardian.apply(lambda x: gop_func(x), axis=1)
# change data type for total votes
guardian['total_votes_2012'] = guardian['TOTAL VOTES CAST'].astype('float64')
# change fips header name
guardian = guardian.rename(columns={'FIPS Code':'combined_fips'})
# filter out state-level rows
guardian = guardian[guardian['combined_fips'] != '0']

guardian.head()

,State Postal,combined_fips,County Name,TOTAL VOTES CAST,Party,Votes,Party.1,Votes.1,Party.2,Votes.2,Party.3,Votes.3,Party.4,Votes.4,votes_dem_2012,votes_gop_2012,total_votes_2012
1,AK,2000,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN,91696.0,121234.0,220596.0
3,AL,1001,Autauga,23909,Dem,6354,GOP,17366,Ind,31.0,Ind,136.0,Ind,22.0,6354.0,17366.0,23909.0
4,AL,1003,Baldwin,84988,Dem,18329,GOP,65772,Ind,122.0,Ind,597.0,Ind,168.0,18329.0,65772.0,84988.0
5,AL,1005,Barbour,11459,Dem,5873,GOP,5539,Ind,9.0,Ind,32.0,Ind,6.0,5873.0,5539.0,11459.0
6,AL,1007,Bibb,8391,Dem,2200,GOP,6131,Ind,13.0,Ind,38.0,Ind,9.0,2200.0,6131.0,8391.0


In [7]:
# drop irrelevant columns
guardian_clean = guardian.drop(labels=['TOTAL VOTES CAST','Votes','Votes.1','Votes.2','Votes.3','Votes.4','Party','Party.1','Party.2','Party.3','Party.4'],axis=1)
# format dataframe headers
guardian_clean.rename(columns={'State Postal':'state_abbr','County Name':'county_name','state_fips_a':'state_fips'},inplace=True)

guardian_clean[guardian_clean['state_abbr'] == 'DC']

,state_abbr,combined_fips,county_name,votes_dem_2012,votes_gop_2012,total_votes_2012
457,DC,11001,District of Columbia,222332.0,17337.0,243348.0


In [8]:
# guardian reports many states at the town-level (ME, for instance)
# roll up to the county_fips level
county_level_pivot = pd.pivot_table(guardian_clean,index=["combined_fips"],values=["votes_dem_2012","votes_gop_2012","total_votes_2012"],aggfunc=np.sum)
county_level_2012 =  county_level_pivot.reset_index()
county_level_2012[county_level_2012['combined_fips'] == '23007'] # roll up to ME county 23007

,combined_fips,total_votes_2012,votes_dem_2012,votes_gop_2012
931,23007,16112.0,9292.0,6305.0


In [9]:
# guardian data combines state- and county-level fips codes into one code
# extract correct county fips codes
county_level_2012['county_fips'] = county_level_2012['combined_fips'].str.extract('(.{3})\s*$',expand=False)
# extract correct state fips codes
# guardian['state_fips'] = ''

state_fips = []
for i,r in county_level_2012['combined_fips'].iteritems():
    if len(r) > 4:
        state_fips.append(r[0:2])
    else:
        state_fips.append('0' + r[0:1])

county_level_2012['state_fips'] = pd.Series(state_fips)

county_level_2012[county_level_2012['combined_fips'] == '23007']

,combined_fips,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips
931,23007,16112.0,9292.0,6305.0,007,23


In [10]:
# add percentages for each R and D of total votes
# calculate percentage of total vote per major candidates
county_level_2012['per_dem_2012'] = county_level_2012['votes_dem_2012'] / county_level_2012['total_votes_2012']
county_level_2012['per_gop_2012'] = county_level_2012['votes_gop_2012'] / county_level_2012['total_votes_2012']
# county_level_2012['diff_2012'] = abs(county_level_2012['votes_gop_2012'] - county_level_2012['votes_dem_2012']).map('{:,.0f}'.format)
county_level_2012['diff_2012'] = abs(county_level_2012['votes_gop_2012'] - county_level_2012['votes_dem_2012'])

# domain for % point difference is set to [-1,1] for visualization purposes
county_level_2012['per_point_diff_2012'] = county_level_2012['per_dem_2012'] - county_level_2012['per_gop_2012']
# view per fips code
county_level_2012[county_level_2012['combined_fips'] == '23007']

,combined_fips,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012
931,23007,16112.0,9292.0,6305.0,007,23,0.576713,0.391323,2987.0,0.18539


## Townhall data

In [11]:
# each page has a summary table that rolls up results at the state level
# get rid of it
def cond(x):
    if x:
        return x.startswith("table ec-table") and not "table ec-table ec-table-summary" in x
    else:
        return False

In [12]:
# list of state abbreviations
states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

# headers for csv export
data = [['state_abbr', 'county_name', 'party', 'votes_total_2016']]

In [13]:
# loop through each state's web page http://townhall.com/election/2016/president/%s/county, where %s is the state abbr
for state in states:
    r = urllib.request.urlopen('http://townhall.com/election/2016/president/' + state + '/county').read()
    soup = BeautifulSoup(r, "html.parser")

    # loop through each <table> tag with .ec-table class
    tables = soup.findAll('table', attrs={'class':cond})

    for table in tables:
        if table.findParent("table") is None:
            table_body = table.find('tbody')

            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                # first tbody tr has four td
                if len(cols) == 4:
                    # strip text from each td
                    divs = cols[0].find_all('div')
                    county = divs[0].text.strip()
                    party = cols[1]['class'][0]
                    total_votes_2016 = int(cols[2].text.strip().replace(',','').replace('-','0'))
                # all other tbody tr have three td
                else:
                    party = cols[1]['class'][0]
                    total_votes_2016 = int(cols[1].text.strip().replace(',','').replace('-','0'))
                    
                #combine each row's results
                rowData = [state,county,party,total_votes_2016]
                data.append(rowData)

In [14]:
townhall = pd.DataFrame(data) # throw results in dataframe
new_header = townhall.iloc[0] #grab the first row for the header
townhall = townhall[1:] #take the data less the header row
townhall.columns = new_header #set the header row as the df header
townhall['votes_total_2016'] = townhall['votes_total_2016'].astype('float64')
print(townhall.shape[0])
townhall.head()

14188


,state_abbr,county_name,party,votes_total_2016
1,AL,Autauga,GOP,18110.0
2,AL,Autauga,DEM,5908.0
3,AL,Autauga,IND,538.0
4,AL,Autauga,IND,105.0
5,AL,Baldwin,GOP,72780.0


In [15]:
# view by state
townhall[(townhall['state_abbr'] == 'DC')]

,state_abbr,county_name,party,votes_total_2016
1415,DC,Washington,DEM,260223.0
1416,DC,Washington,GOP,11553.0
1417,DC,Washington,LIB,4501.0
1418,DC,Washington,DCG,3995.0


In [16]:
# view special cases
townhall[(townhall['state_abbr'] == 'NV') & (townhall['county_name'] == 'Carson City')]

,state_abbr,county_name,party,votes_total_2016
7943,NV,Carson City,GOP,13125.0
7944,NV,Carson City,DEM,9610.0
7945,NV,Carson City,LIB,1159.0
7946,NV,Carson City,OTH,1122.0


In [17]:
# fix townhall county name for Washington DC, Sainte Genevieve, MO, Oglala, SD
townhall.loc[townhall['state_abbr'] =='DC', 'county_name'] = 'District of Columbia'
townhall.loc[townhall['county_name'] == 'Sainte Genevieve', 'county_name'] = 'Ste. Genevieve County'
townhall.loc[townhall['county_name'] == 'Oglala Lakota', 'county_name'] = 'Oglala'
townhall[(townhall['county_name'] == 'District of Columbia') | (townhall['county_name'] == 'Ste. Genevieve County') | (townhall['county_name'] == 'Oglala')]

,state_abbr,county_name,party,votes_total_2016
1415,DC,District of Columbia,DEM,260223.0
1416,DC,District of Columbia,GOP,11553.0
1417,DC,District of Columbia,LIB,4501.0
1418,DC,District of Columbia,DCG,3995.0
7227,MO,Ste. Genevieve County,GOP,5495.0
7228,MO,Ste. Genevieve County,DEM,2540.0
7229,MO,Ste. Genevieve County,LIB,323.0
7230,MO,Ste. Genevieve County,GRN,55.0
7231,MO,Ste. Genevieve County,OTH,46.0
10780,SD,Oglala,DEM,2504.0


In [18]:
# change 'Co.' to 'County' in county_name to match census county name
townhall['county_name'] = townhall['county_name'].apply(lambda x: x.replace('Co.','County').strip())
townhall[(townhall['state_abbr'] == 'NV') & (townhall['county_name'] == 'Carson City')]

,state_abbr,county_name,party,votes_total_2016
7943,NV,Carson City,GOP,13125.0
7944,NV,Carson City,DEM,9610.0
7945,NV,Carson City,LIB,1159.0
7946,NV,Carson City,OTH,1122.0


In [19]:
# combine state and county names to create a unique key to join on other datasets
townhall['combined'] = townhall['state_abbr'] + townhall['county_name'].apply(lambda x: x.replace(' ','').lower())
townhall[(townhall['state_abbr'] == 'DC')]

,state_abbr,county_name,party,votes_total_2016,combined
1415,DC,District of Columbia,DEM,260223.0,DCdistrictofcolumbia
1416,DC,District of Columbia,GOP,11553.0,DCdistrictofcolumbia
1417,DC,District of Columbia,LIB,4501.0,DCdistrictofcolumbia
1418,DC,District of Columbia,DCG,3995.0,DCdistrictofcolumbia


## Census data

In [20]:
# county_fips data from https://www.census.gov/geo/reference/codes/cou.html
census = pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',sep=',',header=None, dtype=str)
census.columns = ['state_abbr', 'state_fips', 'county_fips', 'county_name', 'fips_class_code']
print(census.shape)
census.head()

(3235, 5)


,state_abbr,state_fips,county_fips,county_name,fips_class_code
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [21]:
# view by state
ak_counties = census[(census['state_abbr'] == 'AK')].shape[0]
print(ak_counties)

29


In [22]:
# veiw special cases
census[(census['state_abbr'] == 'NV') & (census['county_name'] == 'Carson City')]

,state_abbr,state_fips,county_fips,county_name,fips_class_code
1763,NV,32,510,Carson City,C7


In [23]:
# change Shannon County, SD to Oglala County, SD
# http://rapidcityjournal.com/news/local/it-s-official-oglala-lakota-county-replaces-shannon-county-name/article_ac5c2369-3fea-5f94-9898-b007b7ddf22c.html
# townhall.loc[townhall['county_name'] == 'Sainte Genevieve', 'county_name'] = 'Ste. Genevieve County'
census.loc[(census['county_name'] == 'Shannon County') & (census['state_abbr'] == 'SD'), 'county_name'] = 'Oglala County'
census[(census['state_abbr'] == 'SD')]

,state_abbr,state_fips,county_fips,county_name,fips_class_code
2362,SD,46,003,Aurora County,H1
2363,SD,46,005,Beadle County,H1
2364,SD,46,007,Bennett County,H1
2365,SD,46,009,Bon Homme County,H1
2366,SD,46,011,Brookings County,H1
2367,SD,46,013,Brown County,H1
2368,SD,46,015,Brule County,H1
2369,SD,46,017,Buffalo County,H1
2370,SD,46,019,Butte County,H1
2371,SD,46,021,Campbell County,H1


In [24]:
# state of Alaska reports results at the precinct and state level; no county level data available
# report results as the states level; 
# ugly fix to get townhall results and census counties to work together
# future plan: roll up precinct-level results to the county level
census.loc[(census['state_abbr'] == 'AK'),'county_name'] = 'Alaska'

In [25]:
# change county_name values in townhall data to match 'county_name' values for C7 fips class code cities
# get and transform C7 city names
cities = (census['state_abbr'][(census['fips_class_code'] == 'C7')] + census['county_name'][(census['fips_class_code'] == 'C7')].apply(lambda x: x.replace('city','').replace(' ','').lower()))
cities

1216          MDbaltimore
1597           MOst.louis
1763         NVcarsoncity
2915         VAalexandria
2916            VAbedford
2917            VAbristol
2918         VAbuenavista
2919    VAcharlottesville
2920         VAchesapeake
2921    VAcolonialheights
2922          VAcovington
2923           VAdanville
2924            VAemporia
2925            VAfairfax
2926        VAfallschurch
2927           VAfranklin
2928     VAfredericksburg
2929              VAgalax
2930            VAhampton
2931       VAharrisonburg
2932           VAhopewell
2933          VAlexington
2934          VAlynchburg
2935           VAmanassas
2936       VAmanassaspark
2937       VAmartinsville
2938        VAnewportnews
2939            VAnorfolk
2940             VAnorton
2941         VApetersburg
2942           VApoquoson
2943         VAportsmouth
2944            VAradford
2945           VArichmond
2946            VAroanoke
2947              VAsalem
2948           VAstaunton
2949            VAsuffolk
2950      VA

In [26]:
# loop through 'combined' column and compare to cities series to add 'city' to H1 fips class code to townhall data
for i, row in cities.iteritems():
    if row != 'NVcarsoncity':
        townhall.loc[townhall['combined'] == row, 'combined'] = row + 'city'
    
townhall[(townhall['combined'] == 'NVcarsoncity')]

,state_abbr,county_name,party,votes_total_2016,combined
7943,NV,Carson City,GOP,13125.0,NVcarsoncity
7944,NV,Carson City,DEM,9610.0,NVcarsoncity
7945,NV,Carson City,LIB,1159.0,NVcarsoncity
7946,NV,Carson City,OTH,1122.0,NVcarsoncity


In [27]:
# remove 'county' from 'combined' column of C7 fips class code counties in townhall
townhall['combined'] = townhall['combined'].str.replace('county','')
townhall[(townhall['county_name'] == 'Oglala')]

,state_abbr,county_name,party,votes_total_2016,combined
10780,SD,Oglala,DEM,2504.0,SDoglala
10781,SD,Oglala,GOP,241.0,SDoglala
10782,SD,Oglala,LIB,104.0,SDoglala
10783,SD,Oglala,OTH,47.0,SDoglala


In [28]:
# combine state abbreviation and county name to create a unique key to join with other datasets
census['combined'] = census['state_abbr'] + census['county_name'].apply(lambda x: x.replace('County','').replace('Parish','').replace(' ','').lower())
census[(census['state_abbr'] == 'DC')]

,state_abbr,state_fips,county_fips,county_name,fips_class_code,combined
319,DC,11,001,District of Columbia,H6,DCdistrictofcolumbia


In [29]:
# return sum of votes by state and county
townhall['total_votes_2016'] = townhall['votes_total_2016'].groupby(townhall['combined']).transform('sum')
townhall_counties = townhall.drop('votes_total_2016',axis=1)

# view dataset by selected state
townhall_counties[(townhall_counties['state_abbr'] == 'DC')]

,state_abbr,county_name,party,combined,total_votes_2016
1415,DC,District of Columbia,DEM,DCdistrictofcolumbia,280272.0
1416,DC,District of Columbia,GOP,DCdistrictofcolumbia,280272.0
1417,DC,District of Columbia,LIB,DCdistrictofcolumbia,280272.0
1418,DC,District of Columbia,DCG,DCdistrictofcolumbia,280272.0


## Combine data

In [30]:
# join census and townhall data on the 'combined' column
right = townhall.set_index('combined')
left = census.set_index('combined')

combined = left.join(right, lsuffix='', rsuffix='_r')
combined = combined.reset_index()
print('Joined dataset has ' + str(combined.shape[0]) + ' items')

# view data by selected state
combined[(combined['state_abbr'] == 'DC')]

Joined dataset has 14422 items


,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,state_abbr_r,county_name_r,party,votes_total_2016,total_votes_2016
1559,DCdistrictofcolumbia,DC,11,001,District of Columbia,H6,DC,District of Columbia,DEM,260223.0,280272.0
1560,DCdistrictofcolumbia,DC,11,001,District of Columbia,H6,DC,District of Columbia,GOP,11553.0,280272.0
1561,DCdistrictofcolumbia,DC,11,001,District of Columbia,H6,DC,District of Columbia,LIB,4501.0,280272.0
1562,DCdistrictofcolumbia,DC,11,001,District of Columbia,H6,DC,District of Columbia,DCG,3995.0,280272.0


In [31]:
# scale Alaska by number of counties
combined.loc[(combined['state_abbr'] == 'AK'),'votes_total_2016'] = (combined['votes_total_2016'][combined['state_abbr'] == 'AK']/ak_counties).astype(int)
combined[combined['state_abbr'] == 'AK']

,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,state_abbr_r,county_name_r,party,votes_total_2016,total_votes_2016
0,AKalaska,AK,02,013,Alaska,H1,AK,Alaska,GOP,4497.0,246588.0
1,AKalaska,AK,02,013,Alaska,H1,AK,Alaska,DEM,3207.0,246588.0
2,AKalaska,AK,02,013,Alaska,H1,AK,Alaska,LIB,503.0,246588.0
3,AKalaska,AK,02,013,Alaska,H1,AK,Alaska,GRN,153.0,246588.0
4,AKalaska,AK,02,013,Alaska,H1,AK,Alaska,OTH,142.0,246588.0
5,AKalaska,AK,02,016,Alaska,H5,AK,Alaska,GOP,4497.0,246588.0
6,AKalaska,AK,02,016,Alaska,H5,AK,Alaska,DEM,3207.0,246588.0
7,AKalaska,AK,02,016,Alaska,H5,AK,Alaska,LIB,503.0,246588.0
8,AKalaska,AK,02,016,Alaska,H5,AK,Alaska,GRN,153.0,246588.0
9,AKalaska,AK,02,016,Alaska,H5,AK,Alaska,OTH,142.0,246588.0


In [32]:
# return unique dataset
county_level_combined = combined.drop_duplicates()
print('Combined dataset has ' + str(county_level_combined.shape[0]) + ' total items')
                                                   
# return only D and R results
county_level_combined = county_level_combined[(county_level_combined['party'] == 'GOP') | (county_level_combined['party'] == 'DEM')]
print('Filtered dataset has ' + str(county_level_combined.shape[0]) + ' D and R items')

# flatten dataset by adding votes by R and D columns
county_level_combined['votes_dem_2016'] = county_level_combined['votes_total_2016'].where(county_level_combined['party'] == 'DEM',0).astype('float64')
county_level_combined['votes_gop_2016'] = county_level_combined['votes_total_2016'].where(county_level_combined['party'] == 'GOP',0).astype('float64')

# drop party and party-level totals and other columns
county_level_combined.drop(['party','votes_total_2016','state_abbr_r','county_name_r'], axis=1, inplace=True)

county_level_combined[(county_level_combined['state_abbr'] == 'DC')]

Combined dataset has 14421 total items
Filtered dataset has 6282 D and R items


,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,total_votes_2016,votes_dem_2016,votes_gop_2016
1559,DCdistrictofcolumbia,DC,11,001,District of Columbia,H6,280272.0,260223.0,0.0
1560,DCdistrictofcolumbia,DC,11,001,District of Columbia,H6,280272.0,0.0,11553.0


In [33]:
# pivot data to consolidate
party_pivot = pd.pivot_table(county_level_combined,index=["combined"],values=["votes_gop_2016","votes_dem_2016"],aggfunc=np.sum)
total_pivot = pd.pivot_table(county_level_combined,index=["combined"],values=["total_votes_2016"],aggfunc=np.mean)

# join party and total pivots
combined_pivot = party_pivot.join(total_pivot, lsuffix='', rsuffix='_r')
print('Joined dataset has ' + str(combined_pivot.shape[0]) + ' items')
combined_pivot

# add percentages for each R and D of total votes
# calculate percentage of total vote per major candidates
combined_pivot['per_dem_2016'] = combined_pivot['votes_dem_2016'] / combined_pivot['total_votes_2016']
combined_pivot['per_gop_2016'] = combined_pivot['votes_gop_2016'] / combined_pivot['total_votes_2016']
combined_pivot['diff_2016'] = abs(combined_pivot['votes_gop_2016'] - combined_pivot['votes_dem_2016']).astype(int)
# combined_pivot['per_point_diff_2016'] = abs(combined_pivot['per_dem_2016'] - combined_pivot['per_gop_2016']).map('{:,.2%}'.format)
combined_pivot['per_point_diff_2016'] = combined_pivot['per_dem_2016'] - combined_pivot['per_gop_2016']

# join pivotted and unpivotted data
right = census.set_index('combined')
# left = combined_pivot.set_index('combined')

county_level_2016 = combined_pivot.join(right, lsuffix='', rsuffix='_r')
county_level_2016 = county_level_2016.reset_index()
print('Joined dataset has ' + str(county_level_2016.shape[0]) + ' items')

# create FIPS column for d3.js visualizations
county_level_2016['combined_fips'] = county_level_2016['state_fips'].apply(lambda x: x.lstrip('0')) + county_level_2016['county_fips'].astype('str')
# create another FIPS column for vincent visualizations
county_level_2016['FIPS'] = county_level_2016['state_fips'].astype('str') + county_level_2016['county_fips'].astype('str')

# drop irrelevant columns
county_level_2016 = county_level_2016.drop(['combined','county_fips','state_fips','fips_class_code'], axis=1)
print('2016 dataset has ' + str(county_level_2016.shape[0]) + ' items')

# view data by selected state
county_level_2016[county_level_2016['state_abbr'] == 'DC']

Joined dataset has 3113 items
Joined dataset has 3141 items
2016 dataset has 3141 items


,votes_dem_2016,votes_gop_2016,total_votes_2016,per_dem_2016,per_gop_2016,diff_2016,per_point_diff_2016,state_abbr,county_name,combined_fips,FIPS
316,260223.0,11553.0,280272.0,0.928466,0.041221,248670,0.887245,DC,District of Columbia,11001,11001


In [34]:
# join 2012 and 2016 data
left = county_level_2016.set_index('combined_fips')
right = county_level_2012.set_index('combined_fips')

county_level_final = left.join(right, lsuffix='', rsuffix='_r')
county_level_final = county_level_final.reset_index()
print('Final dataset has ' + str(county_level_final.shape[0]) + ' items')

# view by state
county_level_final[county_level_final['state_abbr'] == 'DC']

Final dataset has 3141 items


,combined_fips,votes_dem_2016,votes_gop_2016,total_votes_2016,per_dem_2016,per_gop_2016,diff_2016,per_point_diff_2016,state_abbr,county_name,FIPS,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012
316,11001,260223.0,11553.0,280272.0,0.928466,0.041221,248670,0.887245,DC,District of Columbia,11001,243348.0,222332.0,17337.0,001,11,0.913638,0.071244,204995.0,0.842394


In [35]:
# gut checks
# view number of non-unique fips codes
a = dict(Counter(county_level_final['combined_fips']))
b = list(a.values())
print([num for num in b if num != 1])

# check each state for consistency between combined, state, and county fips codes
gut_check = county_level_final[['state_abbr','combined_fips','state_fips','county_fips','FIPS']]
gut_check[gut_check['state_abbr'] == 'CA']

[]


,state_abbr,combined_fips,state_fips,county_fips,FIPS
186,CA,6001,06,001,06001
187,CA,6003,06,003,06003
188,CA,6005,06,005,06005
189,CA,6007,06,007,06007
190,CA,6009,06,009,06009
191,CA,6011,06,011,06011
192,CA,6013,06,013,06013
193,CA,6015,06,015,06015
194,CA,6017,06,017,06017
195,CA,6019,06,019,06019


## Export data

In [36]:
county_level_final.to_csv('US_County_Level_Presidential_Results_12-16.csv',sep=',')

## Visualize data

In [37]:
#tutorial on vincent choropleths:
# http://wrobstory.github.io/2013/04/python-maps-choropleth.html
# http://wrobstory.github.io/2013/10/mapping-data-python.html
geo_data = [{'name': 'counties',
             'url': 'https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_counties.topo.json',
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': 'https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_states.topo.json',
             'feature': 'us_states.geo'}
             ]

In [38]:
vis = vincent.Map(data=county_level_final, geo_data=geo_data, scale=1100, projection='albersUsa',
          data_bind='per_point_diff_2012', data_key='FIPS',
          map_key={'counties': 'properties.FIPS'})

#Get rid of State fill, customize stroke color
del vis.marks[1].properties.update

vis.marks[0].properties.enter.stroke_opacity = vincent.ValueRef(value=0.1)
vis.marks[1].properties.enter.stroke.value = '#fff'

vis.to_json('vega.json')

vis.rebind(column='per_point_diff_2012', brew='RdBu')
vis.scales['color'].domain = [-1, 1]
vis.legend(title='2012 US Presidential Election Results by County')
vis.display()

In [39]:
vis = vincent.Map(data=county_level_final, geo_data=geo_data, scale=1100, projection='albersUsa',
          data_bind='per_point_diff_2016', data_key='FIPS',
          map_key={'counties': 'properties.FIPS'})

#Get rid of State fill, customize stroke color
del vis.marks[1].properties.update

vis.marks[0].properties.enter.stroke_opacity = vincent.ValueRef(value=0.1)
vis.marks[1].properties.enter.stroke.value = '#fff'

vis.to_json('vega.json')

vis.rebind(column='per_point_diff_2016', brew='RdBu')
vis.scales['color'].domain = [-1, 1]
vis.legend(title='2016 US Presidential Election Results by County')
vis.display()